# Modules

In [ ]:
from sklearn.datasets import load_iris, make_gaussian_quantiles, make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss, classification_report, confusion_matrix
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.naive_bayes import BernoulliNB, MultinomialNB, ComplementNB


plt.style.use('ggplot')

# Common methods

In [ ]:
def plot_confusion_matrix(y_true, y_pred, labels=["Negative", "Positive"]):
    """
    Plot confusion matrix.

    Args:
        y_true (array-like): True labels.
        y_pred (array-like): Predicted labels.
        labels (list, optional): List of label names. Defaults to ["Negative", "Positive"].
        show_precision_recall (bool, optional): Whether to show precision and recall. Defaults to True.

    Returns:
        None
    """
    plt.figure(figsize=(4, 4), dpi=100)
    sns.set_style("whitegrid")
    cm = confusion_matrix(y_true, y_pred)

    df_cm = pd.DataFrame(cm, index=labels,
                         columns=["Predicted " + labels[0], "Predicted " + labels[1]])
    sns.heatmap(df_cm, annot=True, fmt='g')

    plt.show()

In [ ]:
def value_distribution(df):
    """
    Print value distribution for each column in the DataFrame.

    Args:
        df (DataFrame): Input DataFrame.

    Returns:
        None
    """
    num_rows = len(df)
    for column in df.columns:
        print("Value distribution for column:", column)
        value_counts = df[column].value_counts()
        percentages = (value_counts / num_rows) * 100
        for value, count, percent in zip(value_counts.index, value_counts, percentages):
            print(f"{value}: {count} ({percent:.2f}%)")
        print("\n")

# Load dataset

In [ ]:
BASE_URL = "/kaggle/input/sms-spam-collection-dataset/"
RANDOM_SEED = 0

In [ ]:
df = pd.read_csv(BASE_URL+"spam.csv", encoding='ISO-8859-1')

In [ ]:
df.head()

# Clean dataset

Converts column values to strings, combines them into a new 'Message' column, and removes original columns, streamlining DataFrame structure.

In [ ]:
# Convertire i valori delle colonne in stringhe, escludendo NaN
for col in df.columns[1:]:
    df[col] = df[col].astype(str).replace('nan', '')

# Unire i contenuti delle colonne dalla seconda fino all'ultima in una nuova colonna
df['Message'] = df.iloc[:, 1:].apply(lambda x: ''.join(x.dropna()), axis=1)

# Rimuovere le colonne di partenza
df.drop(df.columns[1:-1], axis=1, inplace=True)

In [ ]:
df.isnull().sum()

The code maps values in the 'v1' column using a dictionary, converting 'ham' to 0 and 'spam' to 1.







In [ ]:
spam_map = {"ham": 0, "spam": 1}
df["v1"] = df["v1"].map(lambda x: spam_map[x])

In [ ]:
df.rename(columns={'v1': 'Spam'}, inplace=True)
df.head()

In [ ]:
X = df["Message"]
y = df["Spam"]

# Dataset exploration

In [ ]:
# Calcolare il conteggio di spam e non-spam
spam_count = df['Spam'].sum()
non_spam_count = len(df) - spam_count

# Etichette per le fette
labels = ['Spam', 'Non-Spam']

# Dati per il grafico a torta
sizes = [spam_count, non_spam_count]

# Creare il grafico a torta
plt.figure(figsize=(8, 6))
plt.pie(sizes, labels=labels, autopct='%1.1f%%', startangle=140)
plt.title('Spam vs Non-Spam')
plt.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.
plt.show()


In [ ]:
# Calcolare la lunghezza dei messaggi
message_lengths = df['Message'].apply(len)

# Creare l'istogramma
plt.figure(figsize=(10, 6))
plt.hist(message_lengths, bins=50, color='skyblue', edgecolor='black')
plt.title('Messagge length distribution')
plt.xlabel('Messagge length')
plt.ylabel('Frequency')
plt.grid(True)
plt.show()


## Word Cloud
A word cloud displays the most frequent words in the text, sized according to their frequency. The most frequent words appear larger and the less frequent ones smaller.

In [ ]:
from wordcloud import WordCloud

# Create a single string containing all spam messages
spam_messages = ' '.join(df.query('Spam == 1')['Message'])

# Create a word cloud
wordcloud = WordCloud(width=800, height=400, background_color='white').generate(spam_messages)

# Plot the word cloud
plt.figure(figsize=(10, 8))
plt.imshow(wordcloud, interpolation='bilinear')
plt.title('Word Cloud of Spam Messages')
plt.axis('off')
plt.show()


In [ ]:
from wordcloud import WordCloud

# Create a single string containing all spam messages
spam_messages = ' '.join(df.query('Spam == 0')['Message'])

# Create a word cloud
wordcloud = WordCloud(width=800, height=400, background_color='white').generate(spam_messages)

# Plot the word cloud
plt.figure(figsize=(10, 8))
plt.imshow(wordcloud, interpolation='bilinear')
plt.title('Word Cloud of Non-Spam Messages')
plt.axis('off')
plt.show()

In [ ]:
from collections import Counter

# Create a list of all words in the spam messages
spam_words = ' '.join(df.query('Spam == 1')['Message']).split()

# Count the occurrences of each word
word_counts = Counter(spam_words)

# Create a DataFrame with the words and their frequencies
word_counts_df = pd.DataFrame(word_counts.most_common(20), columns=['Word', 'Frequency'])

# Plot the bar chart
plt.figure(figsize=(12, 6))
plt.bar(word_counts_df['Word'], word_counts_df['Frequency'], color='skyblue')
plt.title('Top 20 Most Frequent Words in Spam Messages')
plt.xlabel('Word')
plt.ylabel('Frequency')
plt.xticks(rotation=45)
plt.show()


In [ ]:
from collections import Counter

# Create a list of all words in the spam messages
spam_words = ' '.join(df.query('Spam == 0')['Message']).split()

# Count the occurrences of each word
word_counts = Counter(spam_words)

# Create a DataFrame with the words and their frequencies
word_counts_df = pd.DataFrame(word_counts.most_common(20), columns=['Word', 'Frequency'])

# Plot the bar chart
plt.figure(figsize=(12, 6))
plt.bar(word_counts_df['Word'], word_counts_df['Frequency'], color='coral')
plt.title('Top 20 Most Frequent Words in Non-Spam Messages')
plt.xlabel('Word')
plt.ylabel('Frequency')
plt.xticks(rotation=45)
plt.show()

# TfidfVectorizer

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3, random_state=RANDOM_SEED)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(stop_words="english", max_features=50000)
X_train = tfidf.fit_transform(X_train.tolist())
X_test = tfidf.transform(X_test)
X_train.shape

## Bernoulli Naive Bayes

In [ ]:
benb = BernoulliNB()
benb.fit(X_train, y_train)
report = classification_report(benb.predict(X_test), y_test)
print(report)
plot_confusion_matrix(y_test, benb.predict(X_test))


### SMOTE Version

In [ ]:
from imblearn.over_sampling import SMOTE
X_resampled_smote, y_resampled_smote = SMOTE().fit_resample(X_train, y_train)
benb = BernoulliNB()
benb.fit(X_resampled_smote, y_resampled_smote)
report = classification_report(benb.predict(X_test), y_test)
print(report)
plot_confusion_matrix(y_test, benb.predict(X_test))

### ADASYN version

In [ ]:
from imblearn.over_sampling import ADASYN
X_resampled_adasyn, y_resampled_adasyn = ADASYN().fit_resample(X_train, y_train)
benb = BernoulliNB()
benb.fit(X_resampled_adasyn, y_resampled_adasyn)
report = classification_report(benb.predict(X_test), y_test)
print(report)
plot_confusion_matrix(y_test, benb.predict(X_test))

## Multinomial Naive Bayes

In [ ]:
mnb = MultinomialNB()
mnb.fit(X_train, y_train)
report = classification_report(mnb.predict(X_test), y_test)
print(report)
plot_confusion_matrix(y_test, mnb.predict(X_test))

## Complement Naive Bayes

In [ ]:
comnb = ComplementNB()
comnb.fit(X_train, y_train)
report = classification_report(comnb.predict(X_test), y_test)
print(report)
plot_confusion_matrix(y_test, comnb.predict(X_test))

# Binary CountVectorizer

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3, random_state=RANDOM_SEED)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

# Creare un'istanza di CountVectorizer con binary=True
binary_vectorizer = CountVectorizer(stop_words="english", max_features=1000, binary=True)

# Trasformare i dati di addestramento
X_train_binary = binary_vectorizer.fit_transform(X_train)

# Trasformare i dati di test
X_test_binary = binary_vectorizer.transform(X_test)

## Bernoulli Naive Bayes

In [ ]:
benb = BernoulliNB()
benb.fit(X_train_binary, y_train)
report = classification_report(benb.predict(X_test_binary), y_test)
print(report)
plot_confusion_matrix(y_test, benb.predict(X_test_binary))

### SMOTE Version

In [ ]:
from imblearn.over_sampling import SMOTE
X_resampled_smote, y_resampled_smote = SMOTE().fit_resample(X_train_binary, y_train)
benb = BernoulliNB()
benb.fit(X_resampled_smote, y_resampled_smote)
report = classification_report(benb.predict(X_test_binary), y_test)
print(report)
plot_confusion_matrix(y_test, benb.predict(X_test_binary))

## Multinomial Naive Bayes

In [ ]:
mnb = MultinomialNB()
mnb.fit(X_train_binary, y_train)
report = classification_report(mnb.predict(X_test_binary), y_test)
print(report)
plot_confusion_matrix(y_test, mnb.predict(X_test_binary))

## Complement Naive Bayes

In [ ]:
comnb = ComplementNB()
comnb.fit(X_train_binary, y_train)
report = classification_report(comnb.predict(X_test_binary), y_test)
print(report)
plot_confusion_matrix(y_test, comnb.predict(X_test_binary))